# SSP examples with scmdata interface

This notebook gives an example of running SSP scenarios in FaIR using climate response calibrations from 66 CMIP6 models for a total of 8$\times$66 = 528 ensemble members in parallel.

We will run stochastic mode to attempt to capture internal variability.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scmdata

import cProfile

from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, FAIR, CH4LifetimeMethod
from fair21.defaults import species_config_from_default

In [ ]:
emis_in = scmdata.ScmRun(
    '../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv', 
    lowercase_cols=True
)

In [ ]:
fair_to_scmdata_endpipe_variable_map = {
    'BC': 'BC',
    'CH4': 'CH4',
    'CO': 'CO',
    'CO2_FFI': 'CO2|MAGICC AFOLU',
    'CO2_AFOLU': 'CO2|MAGICC Fossil and Industrial',
    'HFC-125': 'HFC125',
    'HFC-134a': 'HFC134a',
    'HFC-143a': 'HFC143a',
    'HFC-152a': 'HFC152a',
    'HFC-227ea': 'HFC227ea',
    'HFC-23': 'HFC23',
    'HFC-236fa': 'HFC236fa',
    'HFC-245fa': 'HFC245fa',
    'HFC-32': 'HFC32',
    'HFC-365mfc': 'HFC365mfc',
    'HFC-4310mee': 'HFC4310mee',
    'NF3': 'NF3',
    'C2F6': 'C2F6',
    'C3F8': 'C3F8',
    'C4F10': 'C4F10',
    'C5F12': 'C5F12',
    'C6F14': 'C6F14',
    'C7F16': 'C7F16',
    'C8F18': 'C8F18',
    'CF4': 'CF4',
    'cC4F8': 'cC4F8',
    'SF6': 'SF6',
    'SO2F2': 'SO2F2',
    'CCl4': 'CCl4',
    'CFC-11': 'CFC11',
    'CFC-113': 'CFC113',
    'CFC-114': 'CFC114',
    'CFC-115': 'CFC115',
    'CFC-12': 'CFC12',
    'CH2Cl2': 'CH2Cl2',
    'CH3Br': 'CH3Br',
    'CH3Cl3': 'CH3CCl3',
    'CH3Cl': 'CH3Cl',
    'CHCl3': 'CHCl3',
    'HCFC-141b': 'HCFC141b',
    'HCFC-142b': 'HCFC142b',
    'HCFC-22': 'HCFC22',
    'Halon-1202': 'Halon1202',
    'Halon-1211': 'Halon1211',
    'Halon-1301': 'Halon1301',
    'Halon-2402': 'Halon2402',
    'N2O': 'N2O',
    'NH3': 'NH3',
    'NOx': 'NOx',
    'NOx_aviation': 'NOx|MAGICC Fossil and Industrial|Aircraft',
    'OC': 'OC',
    'Sulfur': 'Sulfur',
    'VOC': 'VOC',
}

In [ ]:
scmrun_ssp245 = emis_in.filter(region='World', scenario='ssp245', variable=[f"*|{val}" for val in fair_to_scmdata_endpipe_variable_map.values()])

In [ ]:
# pd.set_option('display.max_rows', 500)
# iwot = 0
# for wot in emis_in.filter(region='World', variable).groupby('model', 'scenario'):
#     iwot = iwot + 1
#     if iwot==12:
#         print(wot)
#         break

In [ ]:
# possible_variables = [
    
# ]

## Define our SpeciesIDs

In this run, we want to include the kitchen sink - so the dict of `SpeciesID`s contains everything possible. When constructing your own scenarios, this could be copied as a starting point.

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'co2_ffi': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'co2_afolu': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'co2': SpeciesID('CO2', Category.CO2),
    'ch4': SpeciesID('CH4', Category.CH4),
    'n2o': SpeciesID('N2O', Category.N2O),
    'cfc-11': SpeciesID('CFC-11', Category.CFC_11),
    'cfc-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'cfc-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'cfc-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'cfc-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'ccl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'chcl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'ch2cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'ch3cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'ch3ccl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'ch3br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'cf4': SpeciesID('CF4', Category.F_GAS),
    'c2f6': SpeciesID('C2F6', Category.F_GAS),
    'c3f8': SpeciesID('C3F8', Category.F_GAS),
    'c-c4f8': SpeciesID('C-C4F8', Category.F_GAS),
    'c4f10': SpeciesID('C4F10', Category.F_GAS),
    'c5f12': SpeciesID('C5F12', Category.F_GAS),
    'c6f14': SpeciesID('C6F14', Category.F_GAS),
    'c7f16': SpeciesID('C7F16', Category.F_GAS),
    'c8f18': SpeciesID('C8F18', Category.F_GAS),
    'hfc-125': SpeciesID('HFC-125', Category.F_GAS),
    'hfc-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'hfc-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'hfc-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'hfc-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'hfc-23': SpeciesID('HFC-23', Category.F_GAS),
    'hfc-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'hfc-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'hfc-32': SpeciesID('HFC-32', Category.F_GAS),
    'hfc-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'hfc-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'nf3': SpeciesID('NF3', Category.F_GAS),
    'sf6': SpeciesID('SF6', Category.F_GAS),
    'so2f2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'bc': SpeciesID('BC', Category.BC),
    'oc': SpeciesID('OC', Category.OC),
    'nh3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'voc': SpeciesID('VOC', Category.SLCF_OZONE_PRECURSOR),
    'co': SpeciesID('CO', Category.SLCF_OZONE_PRECURSOR),
    'nox': SpeciesID('NOx', Category.SLCF_OZONE_PRECURSOR),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'nox_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'lapsi': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'h2o_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

## Grab emissions from SSPs

We use Zeb and Jared's SSP database to fill in our emissions time series. We'll also build our list of `Scenario`s to put into FaIR.

For this, we need to separate out the species that are emitted from those that aren't but we still want to include in the final scenario.

In [ ]:
# these will use the dict keys from above rather than the actual species names.
# Remember, order is not important, but we need to have consistency between the order the species are input here and in the
# SpeciesConfigs later on.

# We'll use mixed-case here, because we want to extract these species from the RCMIP database and that's how they're 
# entered there.
emitted_species = [
    'CO2_FFI', 'CO2_AFOLU', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18',
    'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc',
    'HFC-4310mee', 'NOx_aviation']
forced_species = ['solar', 'volcanic']
from_other_species = ['co2', 'ari', 'aci', 'ozone', 'contrails', 'lapsi', 'h2o_stratospheric', 'land_use']

species_to_include = emitted_species + forced_species + from_other_species

In [ ]:
scenarios_to_include = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']
scenarios = []

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')

### Fill in emissions

In [ ]:
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1750':'2100'
        ].interpolate(axis=1).values.squeeze()

        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species.lower()], emissions=emis_in))
        
    # solar and volcanic forcing still a little hacky
    solar_forcing = np.zeros(351)
    solar_forcing[:270] = df_forc['solar'].values
    volcanic_forcing = np.zeros(351)
    volcanic_forcing[:270] = df_forc['volcanic'].values
    list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
    list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    scenarios.append(Scenario(scenario, list_of_species))

## Fill in Configs

- Grab ClimateResponse configs from calibrations
- use default SpeciesConfigs

In [ ]:
df = pd.read_csv("../data/calibration/4xCO2_cummins.csv")
models = df['model'].unique()

params = {}

configs = []

seedgen = 1355763
for imodel, model in enumerate(models):
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        config_name = f"{model}_{run}"
        climate_response = ClimateResponse(
            ocean_heat_capacity = df.loc[condition, 'C1':'C3'].values.squeeze(),
            ocean_heat_transfer = df.loc[condition, 'kappa1':'kappa3'].values.squeeze(),
            deep_ocean_efficacy = df.loc[condition, 'epsilon'].values[0],
            gamma_autocorrelation = df.loc[condition, 'gamma'].values[0],
            sigma_eta = df.loc[condition, 'sigma_eta'].values[0],
            sigma_xi = df.loc[condition, 'sigma_xi'].values[0],
            stochastic_run = True,
            seed = seedgen
        )
        seedgen = seedgen+399
        species_config = [species_config_from_default(species) for species in species_to_include]
        configs.append(Config(config_name, climate_response, species_config))

## Run FaIR

In [ ]:
start = time.time()
fair = FAIR(scenarios, configs)
#cProfile.run("fair.run()", sort="cumulative")
fair.run()
end = time.time()
print (f"{len(scenarios) * len(configs)} ensemble members in {end - start}s.")

## Make some nice plots

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        np.max(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 5, axis=1), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 16, axis=1), 
        np.percentile(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.temperature[100:, i, :, 0, 0]-fair.temperature[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1850,2100)
    ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios_to_include[i])
pl.suptitle('Temperature anomaly')

In [ ]:
fair.calculate_ocean_heat_content_change()

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        np.max(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 5, axis=1), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 16, axis=1), 
        np.percentile(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.ocean_heat_content_change[100:, i, :, 0, 0]-fair.ocean_heat_content_change[100:151, i, :, 0, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1850,2100)
    #ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios_to_include[i])

pl.suptitle('Ocean heat content change')

In [ ]:
# Squirrel this away as a TODO to check ozone forcing
fair.forcing_array[264, 2, :, 54, 0]*47/37

In [ ]:
fair.species_index_mapping

In [ ]:
scenarios[0].name

In [ ]:
fair.scenarios[0].list_of_species[51].species_id.name

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.min(fair.scenarios[i].list_of_species[51].concentration[100:, :], axis=1), 
        np.max(fair.scenarios[i].list_of_species[51].concentration[100:, :], axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 5, axis=1), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        np.arange(1850.5, 2101), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 16, axis=1), 
        np.percentile(fair.scenarios[i].list_of_species[51].concentration[100:, :], 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        np.arange(1850.5, 2101), 
        np.median(fair.scenarios[i].list_of_species[51].concentration[100:, :], axis=1), 
        color='#000000',
    )
    ax[i//4,i%4].set_xlim(1850,2100)
    ax[i//4,i%4].set_title(scenarios_to_include[i])
pl.suptitle('CO2 concentrations')

In [ ]:
pl.plot(fair.toa_imbalance[:,7,0,0,0])

In [ ]:
pl.plot(fair.forcing_sum_array[:,7,0,0,0])
fair.forcing_sum_array[-1,7,0,0,0]

In [ ]:
fair.forcing_sum_array[270,7,0,0,0]

In [ ]:
pl.plot(fair.temperature[:, 7, 0, 0, 0])
pl.plot(fair.temperature[:, 7, 0, 0, 1])
pl.plot(fair.temperature[:, 7, 0, 0, 2])